##### Copyright 2019 The TensorFlow Neural Structured Learning Authors

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用对抗正则化实现图像分类

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a> </td>
</table>

## 概述

在本教程中，我们将了解如何使用对抗学习 ([Goodfellow et al., 2014](https://arxiv.org/abs/1412.6572)) 来实现使用神经结构学习 (NSL) 框架的图像分类。

对抗学习的核心思想是，除了有机训练数据外，还将使用对抗性扰动数据（称为对抗样本）来训练模型。在人眼看来，这些对抗样本看起来与原始样本相同，但扰动会导致模型混淆并做出错误的预测或分类。对抗样本旨在有意误导模型做出错误的预测或分类。通过使用此类样本进行训练，模型将学习以具备在预测时能够免受对抗扰动影响的鲁棒性。

在本教程中，我们以如下几个步骤展示了应用对抗学习以获得使用神经结构学习框架的鲁棒模型：

1. 作为基础模型创建神经网络。在本教程中，基础模型是使用 `tf.keras` 函数式 API 创建的；此过程也与使用 `tf.keras` 序列式和子类化 API 创建的模型兼容。有关 TensorFlow 中的 Keras 模型的更多信息，请参阅此[文档](https://tensorflow.google.cn/api_docs/python/tf/keras/Model)。
2. 使用 NSL 框架提供的 **`AdversarialRegularization`** 封装容器类封装基础模型，以创建新的 `tf.keras.Model` 实例。这个新模型将包含对抗损失，作为其训练目标中的一个正则化项。
3. 将训练数据中的样本转换为特征字典。
4. 训练并评估新模型。

## 初学者回顾

TensorFlow 神经结构化学习 YouTube 系列的图像分类部分提供了关于对抗学习的相应[视频解释](https://youtu.be/Js2WJkhdU7k)。下面，我们总结了此视频中解释的关键概念，扩展了上面“概述”部分中提供的解释。

NSL 框架联合优化图像特征和结构化信号，以帮助神经网络更好地学习。但是，如果没有可用于训练神经网络的显式结构，该怎么办？本教程解释了一种涉及创建对抗近邻（从原始样本修改）以动态构造结构的方式。

首先，对抗近邻被定义为样本图像的修改版本，此版本应用了一些小的扰动，会误导神经网络输出不准确的分类。这些精心设计的扰动通常基于反向梯度方向，目的是在训练期间混淆神经网络。人类可能无法分辨样本图像与其生成的对抗近邻之间的区别。不过，对于神经网络而言，应用的扰动在导致不准确的结论方面是有效的。

生成的对抗近邻随后会连接到样本，由此动态地逐个边缘构造结构。使用这种连接，神经网络可学会保持样本和对抗近邻之间的相似度，同时避免错误分类导致的混淆，从而提高整体神经网络的质量和准确率。

下面的代码段是对所涉及步骤的简要解释，而本教程的其余部分将从技术性上进一步深入探究。

1. 读取并准备数据。加载 MNIST 数据集并将特征值归一化以保持在 [0,1] 范围内

```
import neural_structured_learning as nsl

(x_train, y_train), (x_train, y_train) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
```

1. 构建神经网络。本示例使用了 Keras 序贯基础模型。

```
model = tf.keras.Sequential(...)
```


1. 配置对抗模型。包括超参数：应用于对抗正则化的乘数，根据经验选择不同的步长/学习率值。使用围绕已构造神经网络的封装容器类调用对抗正则化。

```
adv_config = nsl.configs.make_adv_reg_config(multiplier=0.2, adv_step_size=0.05)
adv_model = nsl.keras.AdversarialRegularization(model, adv_config)
```

1. 以标准 Keras 工作流程结束：编译、拟合、评估。

```
adv_model.compile(optimizer='adam', loss='sparse_categorizal_crossentropy', metrics=['accuracy'])
adv_model.fit({'feature': x_train, 'label': y_train}, epochs=5)
adv_model.evaluate({'feature': x_test, 'label': y_test})
```

您在此处看到的是通过 2 个步骤和 3 行简单代码实现的对抗学习。这就是神经结构化学习框架的简单性。在下面的部分中，我们将扩展此过程。

## 设置

安装 Neural Structured Learning 软件包。

In [ ]:
!pip install --quiet neural-structured-learning

导入库。我们将 `neural_structured_learning` 缩写为 `nsl`。

In [ ]:
import matplotlib.pyplot as plt
import neural_structured_learning as nsl
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## 超参数

我们收集并解释用于模型训练和评估的超参数（在 `HParams` 对象中）。

输入/输出：

- **`input_shape`**：输入张量的形状。每个图像均为 28 x 28 像素，带有 1 个通道。
- **`num_classes`**：共有 10 个类，对应于 10 个数字 [0-9]。

模型架构：

- **`conv_filters`**：一个数字列表，其中的各个数字指定卷积层中筛选器的数量。
- **`kernel_size`**：二维卷积窗的大小，由所有卷积层共享。
- **`pool_size`**：在每个最大池化层中缩小图像的缩放系数。
- **`num_fc_units`**：每个全连接层的单元数（即宽度）。

训练和评估：

- **`batch_size`**：用于训练和评估的批次大小。
- **`epochs`**：训练周期数。

对抗学习：

- **`adv_multiplier`**：训练目标中的对抗损失相对于带标签损失的权重。
- **`adv_step_size`**：对抗扰动的幅度。
- **`adv_grad_norm`**：衡量对抗扰动幅度的范数。


In [ ]:
class HParams(object):
  def __init__(self):
    self.input_shape = [28, 28, 1]
    self.num_classes = 10
    self.conv_filters = [32, 64, 64]
    self.kernel_size = (3, 3)
    self.pool_size = (2, 2)
    self.num_fc_units = [64]
    self.batch_size = 32
    self.epochs = 5
    self.adv_multiplier = 0.2
    self.adv_step_size = 0.2
    self.adv_grad_norm = 'infinity'

HPARAMS = HParams()

## MNIST 数据集

[MNIST 数据集](http://yann.lecun.com/exdb/mnist/)包含手写数字（从“0”到“9”）的灰度图像。每个图像均以低分辨率（28 x 28 像素）显示一个数字。涉及的任务是将图像分为 10 个类别，每个数字表示一个类别。

在这里，我们将从 [TensorFlow 数据集](https://tensorflow.google.cn/datasets)加载 MNIST 数据集。它可用于下载数据和构造 `tf.data.Dataset`。加载的数据集有两个子集：

- `train`，包含 60,000 个样本，以及
- `test`，包含 10,000 个样本。

两个子集中的样本使用以下两个键存储在特征字典中：

- `image`：像素值的数组，范围为 0 到 255。
- `label`：真实值标签，范围为 0 到 9。

In [ ]:
datasets = tfds.load('mnist')

train_dataset = datasets['train']
test_dataset = datasets['test']

IMAGE_INPUT_NAME = 'image'
LABEL_INPUT_NAME = 'label'

为了使模型数值稳定，我们通过基于 `normalize` 函数映射数据集来将像素值标归一化至 [0, 1]。打乱训练集顺序并进行批处理后，我们将样本转换为特征元组 `(image, label)` 用于训练基础模型。我们还提供了将元组转换为字典以供后续使用的函数。

In [ ]:
def normalize(features):
  features[IMAGE_INPUT_NAME] = tf.cast(
      features[IMAGE_INPUT_NAME], dtype=tf.float32) / 255.0
  return features

def convert_to_tuples(features):
  return features[IMAGE_INPUT_NAME], features[LABEL_INPUT_NAME]

def convert_to_dictionaries(image, label):
  return {IMAGE_INPUT_NAME: image, LABEL_INPUT_NAME: label}

train_dataset = train_dataset.map(normalize).shuffle(10000).batch(HPARAMS.batch_size).map(convert_to_tuples)
test_dataset = test_dataset.map(normalize).batch(HPARAMS.batch_size).map(convert_to_tuples)

## 基础模型

我们的基础模型将是一个由 3 个卷积层和 2 个全连接层构成的神经网络（如 `HPARAMS` 中定义）。在这里，我们使用 Keras 函数式 API 对其进行定义。请随意尝试其他 API 或模型架构（例如子类化）。请注意，NSL 框架确实支持全部三种类型的 Keras API。

In [ ]:
def build_base_model(hparams):
  """Builds a model according to the architecture defined in `hparams`."""
  inputs = tf.keras.Input(
      shape=hparams.input_shape, dtype=tf.float32, name=IMAGE_INPUT_NAME)

  x = inputs
  for i, num_filters in enumerate(hparams.conv_filters):
    x = tf.keras.layers.Conv2D(
        num_filters, hparams.kernel_size, activation='relu')(
            x)
    if i < len(hparams.conv_filters) - 1:
      # max pooling between convolutional layers
      x = tf.keras.layers.MaxPooling2D(hparams.pool_size)(x)
  x = tf.keras.layers.Flatten()(x)
  for num_units in hparams.num_fc_units:
    x = tf.keras.layers.Dense(num_units, activation='relu')(x)
  pred = tf.keras.layers.Dense(hparams.num_classes)(x)
  model = tf.keras.Model(inputs=inputs, outputs=pred)
  return model

In [ ]:
base_model = build_base_model(HPARAMS)
base_model.summary()

接下来，我们训练并评估基础模型。

In [ ]:
base_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc'])
base_model.fit(train_dataset, epochs=HPARAMS.epochs)

In [ ]:
results = base_model.evaluate(test_dataset)
named_results = dict(zip(base_model.metrics_names, results))
print('\naccuracy:', named_results['acc'])

我们可以看到，基础模型在测试集上获得了 99% 的准确率。我们将在下面的[对抗扰动下的鲁棒性](#scrollTo=HXK9MGG8lBX3)部分中了解其鲁棒性。

## 对抗正则化模型

在这里，我们将展示如何在 NSL 框架下使用短短几行代码将对抗训练整合到 Keras 模型中。包装基础模型以创建新的 `tf.Keras.Model`，其训练目标包括对抗正则化。

首先，我们使用辅助函数 `nsl.configs.make_adv_reg_config` 创建具有所有相关超参数的配置对象。

In [ ]:
adv_config = nsl.configs.make_adv_reg_config(
    multiplier=HPARAMS.adv_multiplier,
    adv_step_size=HPARAMS.adv_step_size,
    adv_grad_norm=HPARAMS.adv_grad_norm
)

现在，我们可以使用 `AdversarialRegularization` 包装基础模型。在这里，我们将创建一个新的基础模型 (`base_adv_model`)，以便现有基础模型 (`base_model`) 可在后续比较中使用。

返回的 `adv_model` 为 `tf.keras.Model` 对象，其训练目标包括对抗损失的正则化项。为了计算该损失，除了常规输入（特征 `image`）之外，模型还必须有权访问标签信息（特征 `label`）。为此，我们将数据集中的样本从元组转换回字典。然后，我们通过 `label_keys` 参数告知模型哪项特征包含标签信息。

In [ ]:
base_adv_model = build_base_model(HPARAMS)
adv_model = nsl.keras.AdversarialRegularization(
    base_adv_model,
    label_keys=[LABEL_INPUT_NAME],
    adv_config=adv_config
)

train_set_for_adv_model = train_dataset.map(convert_to_dictionaries)
test_set_for_adv_model = test_dataset.map(convert_to_dictionaries)

接下来，我们编译、训练并评估对抗正则化模型。可能会出现诸如“Output missing from loss dictionary”之类的警告，该警告问题不大，因为 `adv_model` 不依赖于计算总损失的基础实现。

In [ ]:
adv_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc'])
adv_model.fit(train_set_for_adv_model, epochs=HPARAMS.epochs)

In [ ]:
results = adv_model.evaluate(test_set_for_adv_model)
named_results = dict(zip(adv_model.metrics_names, results))
print('\naccuracy:', named_results['sparse_categorical_accuracy'])

可以看到，对抗正则化模型在测试集上的表现也非常好（准确率达 99%）。

## 对抗扰动下的鲁棒性

现在，我们将比较基础模型和对抗正则化模型在对抗扰动下的鲁棒性。

我们将使用 `AdversarialRegularization.perturb_on_batch` 函数来生成对抗性扰动样本。我们希望基于基础模型生成样本。为此，我们使用 `AdversarialRegularization` 来包装基础模型。请注意，只要不调用训练 (`Model.fit`)，模型中的学习变量就不会更改，并且该模型仍为[基础模型](#scrollTo=JrrMpPNmpCKK)部分中的同一个模型。

In [ ]:
reference_model = nsl.keras.AdversarialRegularization(
    base_model, label_keys=[LABEL_INPUT_NAME], adv_config=adv_config)
reference_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc'])

我们在字典中收集要评估的模型，并为每个模型创建一个指标对象。

请注意，我们采用 `adv_model.base_model`，以便具有与基础模型相同的输入格式（不需要标签信息）。`adv_model.base_model` 中的学习变量与 `adv_model` 的相同。

In [ ]:
models_to_eval = {
    'base': base_model,
    'adv-regularized': adv_model.base_model
}
metrics = {
    name: tf.keras.metrics.SparseCategoricalAccuracy()
    for name in models_to_eval.keys()
}

这是生成扰动样本并使用它们评估模型的循环。我们将在下一部分中保存扰动图像、标签和预测以供可视化。

In [ ]:
perturbed_images, labels, predictions = [], [], []

for batch in test_set_for_adv_model:
  perturbed_batch = reference_model.perturb_on_batch(batch)
  # Clipping makes perturbed examples have the same range as regular ones.
  perturbed_batch[IMAGE_INPUT_NAME] = tf.clip_by_value(
      perturbed_batch[IMAGE_INPUT_NAME], 0.0, 1.0)
  y_true = perturbed_batch.pop(LABEL_INPUT_NAME)
  perturbed_images.append(perturbed_batch[IMAGE_INPUT_NAME].numpy())
  labels.append(y_true.numpy())
  predictions.append({})
  for name, model in models_to_eval.items():
    y_pred = model(perturbed_batch)
    metrics[name](y_true, y_pred)
    predictions[-1][name] = tf.argmax(y_pred, axis=-1).numpy()

for name, metric in metrics.items():
  print('%s model accuracy: %f' % (name, metric.result().numpy()))

可以看到，当输入遭遇对抗性扰动时，基础模型的准确率会急剧下降（从 99％ 降至约 50％）。另一方面，对抗正则化模型的准确率仅小幅降低（从 99％ 降至 95％）。这证明了对抗学习对于提高模型的鲁棒性有着显著效果。

## 对抗性扰动图像示例

在这里，我们查看一下对抗性扰动图像。可以看到，扰动图像仍显示人类可以识别的数字，但是可以成功地欺骗基础模型。

In [ ]:
batch_index = 0

batch_image = perturbed_images[batch_index]
batch_label = labels[batch_index]
batch_pred = predictions[batch_index]

batch_size = HPARAMS.batch_size
n_col = 4
n_row = (batch_size + n_col - 1) // n_col

print('accuracy in batch %d:' % batch_index)
for name, pred in batch_pred.items():
  print('%s model: %d / %d' % (name, np.sum(batch_label == pred), batch_size))

plt.figure(figsize=(15, 15))
for i, (image, y) in enumerate(zip(batch_image, batch_label)):
  y_base = batch_pred['base'][i]
  y_adv = batch_pred['adv-regularized'][i]
  plt.subplot(n_row, n_col, i+1)
  plt.title('true: %d, base: %d, adv: %d' % (y, y_base, y_adv))
  plt.imshow(tf.keras.utils.array_to_img(image), cmap='gray')
  plt.axis('off')

plt.show()

## 结论

我们演示了如何使用对抗学习来实现利用神经结构学习 (NSL) 框架的图像分类。我们鼓励用户（在超参数中）尝试使用不同的对抗设置，并观察它们如何影响模型的鲁棒性。